# Implementation of Skip-Gram model



While a bag-of-words model predicts a word given the neighboring context, a skip-gram model predicts the context (or neighbors) of a word, given the word itself. The model is trained on skip-grams, which are n-grams that allow tokens to be skipped (see the diagram below for an example). The context of a word can be represented through a set of skip-gram pairs of `(target_word, context_word)` where `context_word` appears in the neighboring context of `target_word`.

The training objective of the skip-gram model is to maximize the probability of predicting context words given the target word. For a sequence of words *w<sub>1</sub>, w<sub>2</sub>, ... w<sub>T</sub>*, the objective can be written as the average log probability

![word2vec_skipgram_objective](https://tensorflow.org/text/tutorials/images/word2vec_skipgram_objective.png)

where `c` is the size of the training context. The basic skip-gram formulation defines this probability using the softmax function.

![word2vec_full_softmax](https://tensorflow.org/text/tutorials/images/word2vec_full_softmax.png)

where *v* and *v<sup>'<sup>* are target and context vector representations of words and *W* is vocabulary size.

Computing the denominator of this formulation involves performing a full softmax over the entire vocabulary words, which are often large (10<sup>5</sup>-10<sup>7</sup>) terms.

The [noise contrastive estimation](https://www.tensorflow.org/api_docs/python/tf/nn/nce_loss) (NCE) loss function is an efficient approximation for a full softmax. With an objective to learn word embeddings instead of modeling the word distribution, the NCE loss can be [simplified](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) to use negative sampling.

The simplified negative sampling objective for a target word is to distinguish  the context word from `num_ns` negative samples drawn from noise distribution *P<sub>n</sub>(w)* of words. More precisely, an efficient approximation of full softmax over the vocabulary is, for a skip-gram pair, to pose the loss for a target word as a classification problem between the context word and `num_ns` negative samples.

**Objective of the Skip-gram Model with Negative Sampling**

For a pair consisting of a **target word** $ w_t $ and a **context word** $ w_c $,  
and $ k $ negative samples drawn from the noise distribution $ P_n(w) $,  
the loss function (to be minimized) is:

$$
\mathcal{L} = - \left[
\log \sigma \left( v_{w_c}^\top v_{w_t} \right)
+ \sum_{i=1}^{k} 
\mathbb{E}_{w_i \sim P_n(w)} 
\left[ 
\log \sigma \left( - v_{w_i}^\top v_{w_t} \right)
\right]
\right]
$$

---

Where:

- $ v_{w_t}$: vector of the **target word**  
- $ v_{w_c} $ vector of the **context word**  
- $ v_{w_i} $: vector of a **negative word** sampled from $ P_n(w) $ 
- $ \sigma(x) = \frac{1}{1 + e^{-x}} $: the **sigmoid function**  
- $ k = \text{num\_ns} $: number of negative samples  
- $ P_n(w) $: **noise distribution**

---

Intuitive interpretation:

- The first term $ \log \sigma(v_{w_c}^\top v_{w_t}) $ **maximizes the similarity** between words that appear in the **true context**.  
- The second term $ \sum_{i=1}^{k} \log \sigma(-v_{w_i}^\top v_{w_t}) $ **minimizes the similarity** with **negative (noise) words**.  
- Therefore, the model learns to **distinguish real contexts from false ones**.


## Setup

In [2]:
# Now we have our own vocabulary words we will now implement a skip-gram model using TensorFlow and Keras.
# project the words into a vector space where similar words are closer together using K-means clustering.

import io
import re
import string
import tqdm

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import spacy




/Users/lioula/User/PFE/Make.Org/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [4]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

## Prepare training data for word2vec

In [5]:
# ================================
# CONFIGURATION
# ================================
INPUT = "makeorg_biodiversite.xlsx"
OUTPUT = "output.xlsx"
TEXT_COL = "Proposition"
NEW_COL = "Tokens_lemmas"
MODEL = "fr_core_news_sm"

# ================================
# LOAD MODEL (auto-download if missing)
# ================================
try:
    nlp = spacy.load(MODEL)
except OSError:
    print(f"Downloading {MODEL} ...")
    from spacy.cli import download
    download(MODEL)
    nlp = spacy.load(MODEL)

# ================================
# READ DATA
# ================================
df = pd.read_excel(INPUT)

if TEXT_COL not in df.columns:
    raise ValueError(f"Column '{TEXT_COL}' not found. Available: {list(df.columns)}")

# Optional: remove leading “Il faut …”
df[TEXT_COL] = df[TEXT_COL].str.replace(r'^\s*Il faut\s+', '', case=False, regex=True)

# ================================
# DEFINE CLEANING FUNCTION
# ================================
def doc_to_lemmas(doc):
    """
    Clean and lemmatize text automatically using spaCy:
    - Removes punctuation, numbers, and stopwords (articles, pronouns, etc.)
    - Keeps meaningful lemmas (nouns, verbs, adjectives, adverbs)
    """
    lemmas = []
    for tok in doc:
        if tok.is_punct or tok.is_space:
            continue
        if tok.is_stop:  # automatic stopword detection 
            continue
        if tok.like_num or any(ch.isdigit() for ch in tok.text):
            continue
        if tok.is_alpha or any(ch.isdigit() for ch in tok.text):
            lemma = tok.lemma_.lower()
            lemmas.append(lemma)
    return lemmas

# ================================
# PROCESS TEXTS
# ================================
texts = df[TEXT_COL].astype(str).fillna("").tolist()
docs = nlp.pipe(texts, disable=["parser", "ner"])  # faster

df[NEW_COL] = [doc_to_lemmas(doc) for doc in docs]

# ================================
# SAVE RESULT
# ================================
df.to_excel(OUTPUT, index=False)

# show first few samples
print(df[[TEXT_COL, NEW_COL]].head(10))

                                         Proposition  \
0  favoriser le reboisement urbain afin de baisse...   
1  stopper l’érosion des haies (-23 500km par an)...   
2  stopper l&apos;importation de produits agricol...   
3  réintégrer les arbres et les haies dans l’agri...   
4  empêcher l&apos;importation de produits agrico...   
5  replanter des haies pour éviter l&apos;érosion...   
6  punir sévèrement ceux qui déversent des déchet...   
7  replanter des haies autour des champs afin de ...   
8  préserver, restaurer, maintenir en bon état éc...   
9  soutenir une agriculture protectrice du vivant...   

                                       Tokens_lemmas  
0  [favoriser, reboisement, urbain, baisser, temp...  
1  [stopper, érosion, haie, kilomètre, an, puisse...  
2  [stopper, produit, agricole, traiter, produit,...  
3  [réintégrer, arbre, haie, agriculture, produir...  
4  [empêcher, produit, agricole, traiter, produit...  
5  [replanter, haie, éviter, sol, permettre, faun... 

In [6]:
from collections import Counter
from itertools import chain

# Flatten all token lists into one big list
all_tokens = list(chain.from_iterable(df["Tokens_lemmas"]))

# Count frequency of each word
vocab_counter = Counter(all_tokens)

# Vocabulary size = number of unique words
vocab_size = len(vocab_counter)
print(f"📘 Vocabulary size: {vocab_size} unique lemmas")

# Show 20 most frequent words
print("\n Top 20 most frequent words:")
for word, freq in vocab_counter.most_common(20):
    print(f"{word:20s} {freq}")

📘 Vocabulary size: 6033 unique lemmas

 Top 20 most frequent words:
biodiversité         690
interdire            454
faire                328
chasse               315
arrêter              311
nature               311
animal               303
zone                 274
créer                266
protéger             258
espèce               233
limiter              230
ville                218
haie                 210
arbre                208
naturel              208
produit              204
favoriser            202
mettre               201
interdir             198


In [7]:
text_ds = tf.data.Dataset.from_tensor_slices(
    df["Tokens_lemmas"].apply(lambda tokens: " ".join(tokens)).values
)


The TextVectorization layer in TensorFlow/Keras converts raw text into numerical tensors that a model can understand

In [8]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  return tf.strings.lower(input_data) # already cleaned with spaCy


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 6033 # from previous cell
sequence_length = int(df["Tokens_lemmas"].apply(len).quantile(0.90))

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

Call `TextVectorization.adapt` on the text dataset to create vocabulary.


In [9]:
vectorize_layer.adapt(text_ds.batch(1024))

2025-11-06 01:08:40.730206: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Once the state of the layer has been adapted to represent the text corpus, the vocabulary can be accessed with `TextVectorization.get_vocabulary`. This function returns a list of all vocabulary tokens sorted (descending) by their frequency.

In [10]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', np.str_('biodiversité'), np.str_('interdire'), np.str_('faire'), np.str_('chasse'), np.str_('nature'), np.str_('arrêter'), np.str_('animal'), np.str_('zone'), np.str_('créer'), np.str_('protéger'), np.str_('espèce'), np.str_('limiter'), np.str_('ville'), np.str_('haie'), np.str_('naturel'), np.str_('arbre'), np.str_('produit'), np.str_('favoriser')]


In [11]:
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [12]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

### Obtain sequences from the dataset

You now have a `tf.data.Dataset` of integer encoded sentences. To prepare the dataset for training a word2vec model, flatten the dataset into a list of sentence vector sequences. This step is required as you would iterate over each sentence in the dataset to produce positive and negative examples.

Note: Since the `generate_training_data()` defined earlier uses non-TensorFlow Python/NumPy functions, you could also use a `tf.py_function` or `tf.numpy_function` with `tf.data.Dataset.map`.

In [13]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

5554


2025-11-06 01:08:51.350435: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [14]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[  19 1643  125  564  650   14 1331    0    0    0    0] => [np.str_('favoriser'), np.str_('reboisement'), np.str_('urbain'), np.str_('baisser'), np.str_('température'), np.str_('ville'), np.str_('capter'), '', '', '', '']
[ 129 1848   15  836  106 3955 2066  610   64   82    0] => [np.str_('stopper'), np.str_('érosion'), np.str_('haie'), np.str_('kilomètre'), np.str_('an'), np.str_('puisser'), np.str_('levier'), np.str_('transition'), np.str_('écologique'), np.str_('territoire'), '']
[129  18  43 877  18   3  76   0   0   0   0] => [np.str_('stopper'), np.str_('produit'), np.str_('agricole'), np.str_('traiter'), np.str_('produit'), np.str_('interdire'), np.str_('france'), '', '', '', '']
[1625   17   15  160  211   11    2  669 1343    0    0] => [np.str_('réintégrer'), np.str_('arbre'), np.str_('haie'), np.str_('agriculture'), np.str_('produire'), np.str_('protéger'), np.str_('biodiversité'), np.str_('grâce'), np.str_('agroforesterie'), '', '']
[285  18  43 877  18   3  76   0   0   

## Generate training examples from sequences

`sequences` is now a list of int encoded sentences. Just call the `generate_training_data` function defined earlier to generate training examples for the word2vec model. To recap, the function iterates over each word from each sequence to collect positive and negative context words. Length of target, contexts and labels should be the same, representing the total number of training examples.

In [15]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,  #############
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")


100%|██████████| 5554/5554 [00:02<00:00, 2350.50it/s]




targets.shape: (26599,)
contexts.shape: (26599, 5)
labels.shape: (26599, 5)


## Configure the dataset for performance

To perform efficient batching for the potentially large number of training examples, use the `tf.data.Dataset` API. After this step, you would have a `tf.data.Dataset` object of `(target_word, context_word), (label)` elements to train your word2vec model!

In [16]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<_BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


Apply `Dataset.cache` and `Dataset.prefetch` to improve performance:

In [17]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<_PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


## Model and training

Use the [Keras Subclassing API](https://www.tensorflow.org/guide/keras/custom_layers_and_models) to define your word2vec model with the following layers:

* `target_embedding`: A `tf.keras.layers.Embedding` layer, which looks up the embedding of a word when it appears as a target word. The number of parameters in this layer are `(vocab_size * embedding_dim)`.
* `context_embedding`: Another `tf.keras.layers.Embedding` layer, which looks up the embedding of a word when it appears as a context word. The number of parameters in this layer are the same as those in `target_embedding`, i.e. `(vocab_size * embedding_dim)`.
* `dots`: A `tf.keras.layers.Dot` layer that computes the dot product of target and context embeddings from a training pair.
* `flatten`: A `tf.keras.layers.Flatten` layer to flatten the results of `dots` layer into logits.

With the subclassed model, you can define the `call()` function that accepts `(target, context)` pairs which can then be passed into their corresponding embedding layer. Reshape the `context_embedding` to perform a dot product with `target_embedding` and return the flattened result.

In [18]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

## Define loss function and compile model


For simplicity, you can use `tf.keras.losses.CategoricalCrossEntropy` as an alternative to the negative sampling loss. If you would like to write your own custom loss function, you can also do so as follows:

``` python
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)
```

It's time to build your model! Instantiate your word2vec class with an embedding dimension of 128 (you could experiment with different values). Compile the model with the `tf.keras.optimizers.Adam` optimizer.

In [19]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

Also define a callback to log training statistics for TensorBoard:


In [20]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")


Train the model on the `dataset` for some number of epochs:

In [21]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2026 - loss: 1.6093
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7309 - loss: 1.5942
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9012 - loss: 1.5792
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9501 - loss: 1.5545
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9566 - loss: 1.5100
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9501 - loss: 1.4395
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9408 - loss: 1.3529
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9343 - loss: 1.2623
Epoch 9/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9360 - loss: 1.1705
Epoch 10/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9432 - loss: 1.0766
Epoch 11/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9522 - loss: 0.9808
Epoch 12/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9596 - loss: 0.8852


TensorBoard now shows the word2vec model's accuracy and loss:

In [25]:
#docs_infra: no_execute
%tensorboard --logdir logs

Reusing TensorBoard on port 6007 (pid 92676), started 0:14:18 ago. (Use '!kill 92676' to kill it.)

## Embedding lookup and analysis

Obtain the weights from the model using `Model.get_layer` and `Layer.get_weights`. The `TextVectorization.get_vocabulary` function provides the vocabulary to build a metadata file with one token per line.

In [23]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [24]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n") 
out_v.close()
out_m.close()

In [29]:
# =====================================
# Visualisation des embeddings avec K-means + t-SNE
# =====================================

import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import plotly.express as px

# 1️⃣ Extraire les poids et le vocabulaire
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

# Supprimer le token de padding (index 0)
weights = weights[1:]
vocab = vocab[1:]

print(f"Nombre de mots dans le vocabulaire : {len(vocab)}")
print(f"Dimension des embeddings : {weights.shape[1]}")

# 2️⃣ Clustering K-means
n_clusters = 10  # tu peux ajuster (5, 10, 20 selon ton vocabulaire)
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
clusters = kmeans.fit_predict(weights)

# 3️⃣ Réduction de dimension (t-SNE)
print("Réduction des dimensions avec t-SNE (cela peut prendre 1-2 min)...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
reduced = tsne.fit_transform(weights)

# 4️⃣ Construire un DataFrame pour la visualisation
emb_df = pd.DataFrame({
    "x": reduced[:, 0],
    "y": reduced[:, 1],
    "word": vocab,
    "cluster": clusters
})

# 5️⃣ Visualisation interactive avec Plotly
fig = px.scatter(
    emb_df,
    x="x", y="y",
    color="cluster",
    hover_name="word",
    title="Visualisation des embeddings Word2Vec (t-SNE + K-means)",
    width=900, height=700
)
fig.update_traces(marker=dict(size=8, opacity=0.8, line=dict(width=0)))
fig.show()


Nombre de mots dans le vocabulaire : 6032
Dimension des embeddings : 128
Réduction des dimensions avec t-SNE (cela peut prendre 1-2 min)...


/Users/lioula/User/PFE/Make.Org/.venv/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning:

'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Download the `vectors.tsv` and `metadata.tsv` to analyze the obtained embeddings in the [Embedding Projector](https://projector.tensorflow.org/):

In [ ]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass